# 安裝相關套件

In [1]:
!sudo pip install jieba
!sudo pip install BeautifulSoup4

Cleaning up...
Cleaning up...


# 欲抓的網頁列表

In [2]:
urllist = ['http://chahabi77.pixnet.net/blog/post/436715527',
           'http://chahabi77.pixnet.net/blog/post/403682269',
           'http://chahabi77.pixnet.net/blog/post/354943724',
           'http://chahabi77.pixnet.net/blog/post/386442944',
           'http://chahabi77.pixnet.net/blog/post/235296791',
          ]

# 下載網頁並且組成一個檔案

In [3]:
import urllib2
import json
f = open('./pixnet.txt',"w")

for u in urllist:
    line = {}
    response = urllib2.urlopen(u)
    html = response.read()
    html = html.replace('\r','').replace('\n','')
    line['html'] = html
    line['url'] =u
    
    line_str = json.dumps(line)
    
    f.write(line_str+"\r\n")
f.close()

# 我們來看一下實際檔案的內容
[請點即此鏈結](http://localhost:8001/edit/spark_tutorial/pixnet.txt)

# 載入頁面，並進行 json parsing

In [4]:
import json
pixnet = sc.textFile('./pixnet.txt',use_unicode=False).map(
    lambda x : json.loads(x)).map(lambda x : (x['url'],x['html']))

print "URL:", pixnet.first()[0]
print "資料筆數: ", pixnet.count()
print "HTML 前 200 字元:", pixnet.first()[1][:200]


URL: http://chahabi77.pixnet.net/blog/post/436715527
資料筆數:  5
HTML 前 200 字元: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html lang="zh-TW" itemscope itemtype="http://schema.org/Blog"><head prefix="og: http://ogp.me/ns#


## RDD 常用的 Function 如下
* map(funct) - 對 RDD 的裡頭的元素進行處理動作
* mapValues (func) - 排除 Key,只對 RDD 的裡頭的元素進行處理動作
* reduceByKey(func) - 將相同的 Key 裡頭的 Values 給予合併起來
* count() - 計算 RDD 裡頭元素的個數
* filter(func) - 根據 condition 判斷是否需要保留
* first() - 取得 RDD 裡頭的第一個元素

## <span style="color: blue"> 請填入??，來計算"好吃"的次數(多少頁面之中)</span>

In [6]:
count_nummber = pixnet.filter(lambda x : u"好吃" in x[1] ).count()
if count_nummber == 5 : print "你答對了"

你答對了


##<span style="color: blue">請修改以下的程式碼，並計算"好吃"所有出現次數，注意!!不是頁面數</span>
 <span style="color:red">提示:修改 word_count 函式</span>

In [7]:
def word_count(text):
    
    return text.count(u"好吃")

print "好吃出現了",word_count(u"老師好吃好吃好吃好吃！！！！"),"次"

好吃出現了 4 次


In [8]:
pixnet.mapValues(word_count).collect()

[(u'http://chahabi77.pixnet.net/blog/post/436715527', 6),
 (u'http://chahabi77.pixnet.net/blog/post/403682269', 7),
 (u'http://chahabi77.pixnet.net/blog/post/354943724', 17),
 (u'http://chahabi77.pixnet.net/blog/post/386442944', 8),
 (u'http://chahabi77.pixnet.net/blog/post/235296791', 17)]

In [9]:
total_count = pixnet.mapValues(word_count).map(lambda x : x[1]).reduce(lambda x,y: x+y)
if total_count == 59 : print "你答對了"
else : print "答錯了！你的答案是 %d, 正確答案是59" % (total_count)

答錯了！你的答案是 55, 正確答案是59
